### Importing all the useful packages

In [ ]:
using NeuralPDE, Flux, ModelingToolkit, GalacticOptim, Optim, DiffEqFlux, CPUTime
import ModelingToolkit: Interval, infimum, supremum

@parameters x,y
@variables u(..)

Dxx = Differential(x)^2
Dyy = Differential(y)^2
Dy = Differential(y)
Dx = Differential(x)

### Analytic Solution of the Poisson Equation

In [ ]:
analytic_sol_func(x,y) = (sin(pi*x)*sin(pi*y))/(2pi^2)

### Specifying Domains and the boundary conditions

In [ ]:
# 2D PDE
eq  = Dxx(u(x,y)) + Dyy(u(x,y)) ~ -sin(pi*x)*sin(pi*y)

# Boundary conditions
bcs = [u(0,y) ~ 0.0, u(1,y) ~ -sin(pi*1)*sin(pi*y),
       u(x,0) ~ 0.0, u(x,1) ~ -sin(pi*x)*sin(pi*1)]
# Space and time domains
domains = [x ∈ Interval(0.0,1.0),
           y ∈ Interval(0.0,1.0)]

### Discretization and problem formulation

In [ ]:
dim = 2
chain = FastChain(FastDense(dim,16,Flux.σ),FastDense(16,16,Flux.σ),FastDense(16,1))
# Initial parameters of Neural network
initθ = Float64.(DiffEqFlux.initial_params(chain))

# Discretization
dx = 0.05
discretization = PhysicsInformedNN(chain,GridTraining(dx),init_params =initθ)

@named pde_system = PDESystem(eq,bcs,domains,[x,y],[u(x, y)])
prob = discretize(pde_system,discretization)

cb = function (p,l)
    println("Current loss is: $l")
    return false
end
# optimizer
opt = Optim.BFGS()
@time @CPUtime res = GalacticOptim.solve(prob,opt; cb = cb, maxiters=1000)
phi = discretization.phi

### Plotting the results

In [ ]:
using Plots

xs,ys = [infimum(d.domain):dx/10:supremum(d.domain) for d in domains]

u_predict = reshape([first(phi([x,y],res.minimizer)) for x in xs for y in ys],(length(xs),length(ys)))
u_real = reshape([analytic_sol_func(x,y) for x in xs for y in ys], (length(xs),length(ys)))
diff_u = abs.(u_predict .- u_real)

p1 = plot(xs, ys, u_real, linetype=:contourf,title = "analytic");
savefig("analyticPoisson.pdf")
p2 = plot(xs, ys, u_predict, linetype=:contourf,title = "predict");
savefig("predictPoisson.pdf")
p3 = plot(xs, ys, diff_u,linetype=:contourf,title = "error");
savefig("errorPoisson.pdf")
plot(p1,p2,p3)

In [ ]:
surface(xs, ys, u_real,title = "analytic")

In [ ]:
surface(xs, ys, u_predict,title = "predict")

In [ ]:
surface(xs, ys, diff_u,title = "error")